<a href="https://colab.research.google.com/github/Dhawal-Modi/DSC_2024_Space_Jam/blob/main/Task_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#### Data Visualization here

In [ ]:
#### Data concatenation and preparation here
abnormal_df = pd.read_csv("ptbdb_abnormal.csv", header=None)
normal_df = pd.read_csv("ptbdb_normal.csv", header=None)
abnormal_df.rename(columns={'187': "Label"}, inplace=True)
normal_df.rename(columns={'187': "Label"}, inplace=True)
#downsample the abnormal df to a size closer to the normal df
abnormal_df = abnormal_df.sample(n=5000, random_state=1)
total_df = pd.concat([normal_df, abnormal_df])

In [ ]:
#### Model init and training here